In [9]:
import pandas as pd
import json
from copy import deepcopy
from collections import OrderedDict
from custom_functions_scRBA import *

#### Load

In [2]:
df_rxns = pd.read_excel('./scRAM_network.xlsx')
df_rxns.index = df_rxns.id.to_list()
df_rxns = df_rxns[df_rxns.symbolic_coeffs.isnull() == False]

df_keff = pd.read_excel('./keff.xlsx')
df_keff.index = df_keff.rxn_id.to_list()

df_pro = pd.read_excel('../scRBA/curation/prot_stoich_curation_2021-05-21.xlsx')
df_pro.index = df_pro.id.to_list()

df_params = pd.read_excel('../scRBA/input/parameters_info.xlsx')
df_params.index = df_params.param_id.to_list()

#### Intialize storage dictionary

In [3]:
stoich = OrderedDict()

#### Write constant part of the stoichiometry

In [4]:
for i in df_rxns.index:
    rxn_id = df_rxns.id[i]
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[i])
    met_dict = {k:v for k,v in met_dict.items() if k != ''}
    for k,v in met_dict.items():
        if is_number(v):
            stoich["'" + k + "'.'" + rxn_id + "'"] = v

#### Write symbolic part of the stoichiometry

In [5]:
kribo = 10.5; dummyNAA = 405;

In [6]:
for i in df_rxns.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[i])
    met_dict = {k:v for k,v in met_dict.items() if k != '' and is_number(v) == False}
    sym = df_rxns.symbolic_coeffs[i]
    
    for k,v in met_dict.items():
        if v == '-mu/keff':
            v = str(round(-1 / df_keff.keff[i], 12)) + '*mu'
        elif v == '-mu*NAA/kribo':
            pro_id = i[7:]
            if pro_id in ['DUMMYPRO_c', 'DUMMYPRO_m']:
                v = str(round(dummyNAA / kribo, 12)) + '*mu'
            else:
                v = str(round((len(df_pro.sequence[pro_id]) - 1) / kribo, 12)) + '*mu'
        else:
            v = df_params.string[v]
            
        stoich["'" + k + "'.'" + i + "'"] = str(v)

#### Save

For mu < 0.211

In [7]:
with open('./GAMSmodel/sij_lower_0_211.json', 'w') as f: 
    json.dump(stoich, f)

For mu > 0.211

In [20]:
stoich_higher = deepcopy(stoich)
stoich_higher["'BIO-unmodeledProtBiom'.'BIODIL-BIODIL'"] = -0.188002131616
stoich_higher["'BIO-modeledProtBiom_c'.'BIODIL-BIODIL'"] = -0.253186705097
stoich_higher["'BIO-modeledProtBiom_m'.'BIODIL-BIODIL'"] = -0.022098821287
stoich_higher["'MET-mannan_c'.'BIODIL-BIODIL'"] = -0.539010001587
stoich_higher["'MET-13BDglucan_en'.'BIODIL-BIODIL'"] = -0.836427660720
stoich_higher["'MET-16BDglucan_en'.'BIODIL-BIODIL'"] = -0.220112542295
stoich_higher["'MET-chtn_c'.'BIODIL-BIODIL'"] = -0.019775196008
stoich_higher["'MET-glycogen_c'.'BIODIL-BIODIL'"] = -0.403253827394
stoich_higher["'MET-tre_c'.'BIODIL-BIODIL'"] = -0.020213206243
stoich_higher["'BIO-rna_c'.'BIODIL-BIODIL'"] = -0.07219476

In [21]:
with open('./GAMSmodel/sij_higher_0_211.json', 'w') as f: 
    json.dump(stoich_higher, f)